<a href="https://colab.research.google.com/github/Kimame04/tl-land-use-classification/blob/master/tl_land_use_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Land Use Classification via Transfer Learning

This project aims to be able to conduct image classification using existing pre-trainded models. We will evaluate the effectiveness of some popular deep neural networks on a land usage classification problem.

In [ ]:
import numpy as np
import os
import io
import PIL
from PIL import Image
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import folium
import ee
from folium import plugins

## Generating the Google Maps layer

In [ ]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

first = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(ee.Geometry.Point(-70.48, 43.3631))
         .filterDate('2019-01-01', '2019-12-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

map = folium.Map(location=[1.38281, 103.81025], zoom_start=12)

# Add custom basemaps
basemaps['Google Maps'].add_to(map)
basemaps['Google Satellite'].add_to(map)

# Add fullscreen button
plugins.Fullscreen().add_to(map)

# Display the map.
display(map)

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=YJv3172bbta-rorLeTwD8Em7CAy8S-XFTm1ixXCsXsU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj4YFtSuZeMIvAyrmRtPaquXelFSPatfVrtPVGFwTWB9uT6bsKkQnM

Successfully saved authorization token.


In [ ]:
map.save('sg.html')

## Dataset

http://weegee.vision.ucmerced.edu/datasets/landuse.html

### Retrieving dataset

In [ ]:
!git clone https://github.com/Kimame04/tl-land-use-classification.git


fatal: destination path 'tl-land-use-classification' already exists and is not an empty directory.


In [ ]:
!cd tl-land-use-classification/

In [ ]:
current_path = os.getcwd()
for root, dirs, files in os.walk(current_path, topdown=False):
    for name in files:
        print(os.path.join(root, name))
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":
            if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                print("A jpeg file already exists")
            else:
                outputfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                im = Image.open(os.path.join(root, name))
                im.thumbnail(im.size)
                im.save(outputfile, "JPEG", quality=100)

Streaming output truncated to the last 5000 lines.
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse67.jpg
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse09.jpg
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse65.tif
A jpeg file already exists
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse66.tif
A jpeg file already exists
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse41.tif
A jpeg file already exists
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse87.jpg
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse56.jpg
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse06.jpg
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse47.jpg
/content/tl-land-use-classification/dataset/Images/golfcourse/golfcourse35.tif
A jpeg file already exists
/content/tl-land-use-classification/dataset/Images/

In [ ]:
data_dir = 'tl-land-use-classification/dataset/Images'
batch_size = 32
img_height = 256
img_width = 256

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  label_mode='categorical',
  batch_size=batch_size)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  label_mode='categorical',
  batch_size=batch_size)

class_names = train_dataset.class_names

Found 2156 files belonging to 21 classes.
Using 1725 files for training.
Found 2156 files belonging to 21 classes.
Using 431 files for validation.


### Data processing

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

image_batch, label_batch = next(iter(train_dataset))

### Creating the model

In [ ]:
def createModel(base_model,shape):
  feature_batch = base_model(image_batch)

  base_model.trainable = False

  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  feature_batch_average = global_average_layer(feature_batch)


  inputs = tf.keras.Input(shape=shape)
  x = data_augmentation(inputs)
  x = base_model(x, training=False)
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  outputs = tf.keras.layers.Dense(len(class_names),activation='softmax')(x)
  model = tf.keras.Model(inputs, outputs)

  base_learning_rate = 0.0001
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  model.summary()
  return model

### Fitting the model

In [ ]:
initial_epochs = 5
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=54)

In [ ]:
resnet_model = createModel(tf.keras.applications.ResNet50(weights='imagenet',
                                            include_top=False),(256,256,3))

resnet_model.fit(train_dataset,
          epochs=initial_epochs,
          validation_data=validation_dataset,
          validation_freq=1,
          callbacks=[cp_callback])

94773248/94765736 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 21)                43029     
Total params: 23,630,741
Trainable params: 43,029
Non-traina

In [ ]:
mobilenet_model = createModel(tf.keras.applications.MobileNetV2(include_top=False),(256,256,3))

mobilenet_model.fit(train_dataset,
          epochs=initial_epochs,
          validation_data=validation_dataset,
          validation_freq=1,
          callbacks=[cp_callback])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, None, None, 1280)  2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                26901     
Total params: 2,284,885
Trainable params: 26,901
Non-trainable params: 2,257,984
____________________________________________

In [ ]:
inceptionv3_model = createModel(tf.keras.applications.InceptionV3(include_top=False),(256,256,3))
inceptionv3_model.fit(train_dataset,
          epochs=initial_epochs,
          validation_data=validation_dataset,
          validation_freq=1,
          callbacks=[cp_callback])

87916544/87910968 [==============================] - 1s 0us/step
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 21)                43029     
Total params: 21,845,813
Trainable params: 43,029
Non-traina

In [ ]:
vgg16_model = createModel(tf.keras.applications.VGG16(include_top=False),(256,256,3))
vgg16_model.fit(train_dataset,
          epochs=initial_epochs,
          validation_data=validation_dataset,
          validation_freq=1,
          callbacks=[cp_callback])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 21)                10773     
Total params: 14,725,461
Trainable params: 10,773
Non-trainable params: 14,714,688
__________________________________________

In [ ]:
efficientnet_model = createModel(tf.keras.applications.EfficientNetB0(include_top=False),(256,256,3))
efficientnet_model.fit(train_dataset,
          epochs=initial_epochs,
          validation_data=validation_dataset,
          validation_freq=1,
          callbacks=[cp_callback])

16711680/16705208 [==============================] - 1s 0us/step
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                26901     
Total params: 4,076,472
Trainable params: 26,901
Non-trainab

### Saving the model

In [ ]:
!mkdir -p resnet_model
resnet_model.save('resnet_model')
!zip -r /content/resnet_model.zip /content/resnet_model/

In [ ]:
!mkdir -p mobilenet_model
mobilenet_model.save('mobilenet_model')
!zip -r /content/mobilenet_model.zip /content/mobilenet_model/

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: resnet_model/assets


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: mobilenet_model/assets
INFO:tensorflow:Assets written to: inceptionv3_model/assets


In [ ]:
!mkdir -p inceptionv3_model
inceptionv3_model.save('inceptionv3_model')
!zip -r /content/inceptionv3_model.zip /content/inceptionv3_model/

  adding: content/resnet_model/ (stored 0%)
  adding: content/resnet_model/saved_model.pb (deflated 92%)
  adding: content/resnet_model/variables/ (stored 0%)
  adding: content/resnet_model/variables/variables.index (deflated 78%)
  adding: content/resnet_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/resnet_model/assets/ (stored 0%)
  adding: content/resnet_model/keras_metadata.pb (deflated 96%)
  adding: content/mobilenet_model/ (stored 0%)
  adding: content/mobilenet_model/saved_model.pb (deflated 92%)
  adding: content/mobilenet_model/variables/ (stored 0%)
  adding: content/mobilenet_model/variables/variables.index (deflated 75%)
  adding: content/mobilenet_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/mobilenet_model/assets/ (stored 0%)
  adding: content/mobilenet_model/keras_metadata.pb (deflated 96%)
  adding: content/inceptionv3_model/ (stored 0%)
  adding: content/inceptionv3_model/saved_model.pb (deflated 92%)
  

In [ ]:
!mkdir -p vgg16_model
vgg16_model.save('vgg16_model')
!zip -r /content/vgg16_model.zip /content/vgg16_model/

INFO:tensorflow:Assets written to: vgg16_model/assets
  adding: content/vgg16_model/ (stored 0%)
  adding: content/vgg16_model/assets/ (stored 0%)
  adding: content/vgg16_model/keras_metadata.pb (deflated 96%)
  adding: content/vgg16_model/variables/ (stored 0%)
  adding: content/vgg16_model/variables/variables.index (deflated 64%)
  adding: content/vgg16_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/vgg16_model/saved_model.pb (deflated 91%)


In [ ]:
!mkdir -p efficientnet_model
efficientnet_model.save('efficientnet_model')
!zip -r /content/efficientnet_model.zip /content/efficientnet_model/

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: efficientnet_model/assets
  adding: content/efficientnet_model/ (stored 0%)
  adding: content/efficientnet_model/assets/ (stored 0%)
  adding: content/efficientnet_model/keras_metadata.pb (deflated 96%)
  adding: content/efficientnet_model/variables/ (stored 0%)
  adding: content/efficientnet_model/variables/variables.index (deflated 73%)
  adding: content/efficientnet_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/efficientnet_model/saved_model.pb (deflated 91%)


## Loading the saved model

### Testing the model with unseen data

In [ ]:
def testModel(model):
  loss, acc = model.evaluate(test_dataset, verbose=1)
  print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

In [ ]:
resnet_dir = 'tl-land-use-classification/resnet_model'
resnet_model = tf.keras.models.load_model(resnet_dir)
resnet_model.summary()
testModel(resnet_model)

NameError: ignored

In [ ]:
mobilenet_dir = 'tl-land-use-classification/mobilenet_model'
mobilenet_model = tf.keras.models.load_model(mobilenet_dir)
mobilenet_model.summary()
testModel(mobilenet_model)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, None, None, 1280)  2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                26901     
Total params: 2,284,885
Trainable params: 26,901
Non-trainable params: 2,257,984
____________________________________________

In [ ]:
inceptionv3_dir = 'tl-land-use-classification/inceptionv3_model'
inceptionv3_model = tf.keras.models.load_model(inceptionv3_dir)
inceptionv3_model.summary()
testModel(inceptionv3_model)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
inception_v3 (Functional)    (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 21)                43029     
Total params: 21,845,813
Trainable params: 43,029
Non-trainable params: 21,802,784
__________________________________________

### Converting to TensorFlow Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(resnet_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)